In [1]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

import os
path = os.path.abspath(os.getcwd())

state_gov_nm = 'NH'
year_to_predict = 2020

_legtext09_20 ='{}/{}/legtext09_20.corpus'.format(path,state_gov_nm)

_WinList09_20 = '{}/{}/WinList09_20.list'.format(path,state_gov_nm)

_LegOverallData09_20 = '{}/{}/LegOverallData09_20.csv'.format(path,state_gov_nm)

_LegOverEventCLF09_20 = '{}/{}/trained_models/overallevent_clf.joblib'.format(path,state_gov_nm)


In [2]:
#load all of our documents
with open(_legtext09_20,'rb') as f:
    legcorpus = pickle.load(f)
    

#assemble the text data, and remove the part where it says if it passes or not
combined_docs = {} #{bill_id:text}
total = 0
passed = 0

#a more unorthodox method of grabbing the target passage
leg_target = {} #{bill: 0 or 1, where 1 means pass}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        try:
            if doc[2][-5] == 'approved':
                doc[2] = doc[2][:-5]
                passed += 1
                pass_flag = 1
                combined_docs[doc[0]] = " ".join(doc[2])
            else:
                combined_docs[doc[0]] = " ".join(doc[2])
                
        except:
            combined_docs[doc[0]] = " ".join(doc[2])
            pass
                
    leg_target[doc[0]] = pass_flag
    #for now, ignoring the amendments, but it seems like it could do better with the amendments
    #that would go here (i.e. an else statement saying doc[0] in combined_docs)
    
                   
print("\nTotal bills:", total)
print("Passed bills:", passed)
print("Bill passed ratio:", passed/total)


Total bills: 9520
Passed bills: 6
Bill passed ratio: 0.0006302521008403362


In [3]:
#actually, lets just stick with the given passages for now
leg_data = pd.read_csv(_LegOverallData09_20) 

with open(_WinList09_20,'rb') as f:
    win_state = pickle.load(f)
    
    
leg_targ = pd.DataFrame({"every stage passed":win_state}) 


bill_order = []
bill_target = []
bill_texts = []

leg_targ_old = leg_target
leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
        if bill_id in combined_docs:
            bill_texts.append(combined_docs[bill_id])
        else:
            bill_texts.append("")

print(len(bill_texts))    
print(len(bill_target))
print(len(leg_data))


10612
10612
124345


In [4]:
#features are in a dictionary. 

features = leg_data
print(len(leg_targ))
X = pd.DataFrame(features)
X['leg_targ'] = leg_targ
X_all = X.copy()
X_all = X_all.set_index('bill_id')


X = X.drop_duplicates(subset=['title']) #should probably be bill_id
X = X.set_index('bill_id')
X = X.drop(columns=['time_t', 'event'])


bill_order = []
bill_target = []
bill_texts = []

count = 0
for bill_id in X.index:
    if bill_id in combined_docs:
        bill_texts.append(combined_docs[bill_id])
    else:
        bill_texts.append("")
        
print(len(bill_texts))
X['texts'] = bill_texts



124345
8908


In [5]:

X = X.astype({'numeric_date': 'float'}).sort_values('numeric_date', ascending=True)#, ignore_index=True)
X_all = X_all.astype({'numeric_date': 'float'}).sort_values('numeric_date', ascending=True)

X.head()

Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
667269         7807          -1.0          1.0     1141    -1       S   
132686        73465          -1.0          0.0       54    -1     NaN   
804171        13253          -1.0          1.0     1201    -1       H   
327994        83684          -1.0          1.0       92    -1     NaN   
328014        83686          -1.0          0.8       92    -1     NaN   

        bill_type                                              title  \
bill_id                                                                
667269          B  Relative to training of certain board officers...   
132686          R  Requesting guidance from the Environmental Pro...   
804171          B  Prohibiting a town meeting from materially cha...   
327994          R  Requesting an opinion of the justices concerni...   
328014         CR  Honoring the members of the military and intel...   

        proposed_chamber          primary_name primary_id primary_party  \
bill_id                                                                   
667269                 S         Sharon Carson       4775             R   
132686                 S  Margaret Wood Hassan       4797             D   
804171                 H        James Spillane      16677             R   
327994                 S           Jeb Bradley       4760             R   
328014                 H     David Bettencourt       4761             R   

         number_sponsors committee_introduced  month_introduced  leg_targ  \
bill_id                                                                     
667269                 4                 None                -1         0   
132686                 1                 None                -1         0   
804171                11                 None                -1         0   
327994                 3                 None                -1         0   
328014                15                 None                -1         1   

                                                     texts  
bill_id                                                     
667269   sb introduced session senate bill act relative...  
132686                                                      
804171   hb introduced session house bill act prohibiti...  
327994   sr introduced session senate resolution resolu...  
328014   hcr introduced session house concurrent resolu...

In [6]:

X['month_introduced'] = X['month_introduced'].astype(str)
X_all['month_introduced'] = X_all['month_introduced'].astype(str)
X_all.head()

Unnamed: 0  numeric_date  time_t  \
bill_id                                     
667269         7807          -1.0       1   
328014        83687          -1.0       2   
383650        97048          -1.0       6   
383650        97049          -1.0       7   
383650        97050          -1.0       8   

                                                     event  sponsor_vec  \
bill_id                                                                   
667269                                             hearing          1.0   
328014             ought to pass motion adopted voice vote          0.8   
383650                       committee committee amendment          1.0   
383650                                          voice vote          1.0   
383650   ought to pass ought to pass with amendment mot...          1.0   

         session  year chamber bill_type  \
bill_id                                    
667269      1141    -1       S         B   
328014        92    -1     NaN        CR   
383650       546    -1     NaN         B   
383650       546    -1     NaN         B   
383650       546    -1     NaN         B   

                                                     title proposed_chamber  \
bill_id                                                                       
667269   Relative to training of certain board officers...                S   
328014   Honoring the members of the military and intel...                H   
383650   Relative to jury trials in class actions broug...                H   
383650   Relative to jury trials in class actions broug...                H   
383650   Relative to jury trials in class actions broug...                H   

              primary_name primary_id primary_party  number_sponsors  \
bill_id                                                                
667269       Sharon Carson       4775             R                4   
328014   David Bettencourt       4761             R               15   
383650       Gene Chandler       4830             R                2   
383650       Gene Chandler       4830             R                2   
383650       Gene Chandler       4830             R                2   

        committee_introduced month_introduced  leg_targ  
bill_id                                                  
667269                  None               -1         0  
328014                  None               -1         1  
383650                  None               -1         0  
383650                  None               -1         0  
383650                  None               -1         0

In [7]:

X_all.tail()

Unnamed: 0  numeric_date  time_t  \
bill_id                                     
1294120      123848   2020.075269      13   
1294120      123849   2020.075269      14   
1294120      123850   2020.075269      15   
1294120      123844   2020.075269       9   
1294120      123839   2020.075269       4   

                                       event  sponsor_vec  session  year  \
bill_id                                                                    
1294120      retained in committee committee     0.153846     1729  2020   
1294120                         work session     0.153846     1729  2020   
1294120                    executive session     0.153846     1729  2020   
1294120                  referred to finance     0.153846     1729  2020   
1294120  subcommittee committee work session     0.153846     1729  2020   

        chamber bill_type                                              title  \
bill_id                                                                        
1294120       H         B  Relative to a family and medical leave insuran...   
1294120       H         B  Relative to a family and medical leave insuran...   
1294120       H         B  Relative to a family and medical leave insuran...   
1294120       H         B  Relative to a family and medical leave insuran...   
1294120       H         B  Relative to a family and medical leave insuran...   

        proposed_chamber  primary_name primary_id primary_party  \
bill_id                                                           
1294120                H  Mary Wallner       5091             D   
1294120                H  Mary Wallner       5091             D   
1294120                H  Mary Wallner       5091             D   
1294120                H  Mary Wallner       5091             D   
1294120                H  Mary Wallner       5091             D   

         number_sponsors committee_introduced month_introduced  leg_targ  
bill_id                                                                   
1294120               13                 None                1         0  
1294120               13                 None                1         0  
1294120               13                 None                1         0  
1294120               13                 None                1         0  
1294120               13                 None                1         0

In [9]:
#select the data that we actually want to train on
#it is probably better to use primary id in the future instead of primary name

X_new = X[['sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber', 'primary_name','primary_party',
           'number_sponsors','committee_introduced','month_introduced']].copy()                      

X_all_new = X_all[['time_t','sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber'
                   , 'primary_name','primary_party','number_sponsors','committee_introduced','month_introduced']].copy()                      


#amount_train = 7500
#amount_test = 800
amount_test_all = len(X_all[X_all['year']==year_to_predict])
amount_train_all = len(X_all) - amount_test_all

amount_test = len(X[X['year']==year_to_predict])
amount_train = len(X) - amount_test


X_dum = pd.get_dummies(X_new)
X_all_dum = pd.get_dummies(X_all_new)
X_all_dum['time_t'] = X_all_dum['time_t']/10


#dont think we should include the event yet, too much noise
X_all_dum = pd.concat([X_all_dum, pd.get_dummies(X_all['event'])], axis=1)



X_train_p = X_dum.iloc[0:amount_train]#.iloc
y_train = X['leg_targ'].iloc[0:amount_train]#pd.get_dummies()
X_test_p = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = X['leg_targ'].iloc[amount_train:amount_train+amount_test]#pd.get_dummies()

X_train_p_all = X_all_dum.iloc[0:amount_train_all]#.iloc
y_train_all = X_all['leg_targ'].iloc[0:amount_train_all]#pd.get_dummies()
X_test_p_all = X_all_dum.iloc[amount_train_all:amount_train_all+amount_test_all]#.iloc
y_test_all = X_all['leg_targ'].iloc[amount_train_all:amount_train_all+amount_test_all]#pd.get_dummies()



In [10]:


k_select = SelectKBest(f_classif, k=200)
X_train = k_select.fit_transform(X_train_p, y_train)
X_test = k_select.transform(X_test_p)

k_select_all = SelectKBest(f_classif, k=200)
X_train_all = k_select_all.fit_transform(X_train_p_all, y_train_all)
X_test_all = k_select_all.transform(X_test_p_all)


indices = k_select.get_support(indices=True)
indices_all = k_select_all.get_support(indices=True)

X_train_all = pd.DataFrame(data=X_train_all, index=X_train_p_all.index, columns=X_train_p_all.columns[indices_all])
X_test_all = pd.DataFrame(data=X_test_all, index=X_test_p_all.index, columns=X_test_p_all.columns[indices_all])

print(X_train_p_all.iloc[0,indices])
print(sum(y_train_all)/len(y_train_all))
print(sum(y_test_all)/len(y_test_all))



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [1248 1388 1391 1444] are constant.
  UserWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


time_t                                                                      0.1
sponsor_vec                                                                 1.0
number_sponsors                                                             4.0
chamber_H                                                                   0.0
bill_type_A                                                                 0.0
bill_type_B                                                                 1.0
bill_type_CA                                                                0.0
bill_type_CR                                                                0.0
bill_type_JR                                                                0.0
bill_type_R                                                                 0.0
proposed_chamber_H                                                          0.0
primary_name_Amanda Gourgue                                                 0.0
primary_name_Amy Perkins                

In [11]:
mask = X_all_new['committee_introduced'] == 'Commerce, Senate Journal 3, Pg.35'
X_all_new['committee_introduced'][mask].head()

Series([], Name: committee_introduced, dtype: object)

In [12]:
#X_all_new.loc[243547]

In [13]:
y_test_all.head()

bill_id
1294571    0
1294571    0
1294571    0
1294571    0
1294578    0
Name: leg_targ, dtype: int64

In [14]:
compressed_legtext = pd.DataFrame(X['texts'].copy())
# print(len(compressed_legtext))
# compressed_legtext['target'] = ordered_target
# compressed_legtext = compressed_legtext.loc[~compressed_legtext.index.duplicated(keep='first')]
# #compressed_legtext.drop_duplicates(subset=compressed_legtext.index, keep='first')
# #compressed_legtext.drop_duplicates(keep='first') 
# print(len(compressed_legtext))
compressed_legtext.head()

texts
bill_id                                                   
667269   sb introduced session senate bill act relative...
132686                                                    
804171   hb introduced session house bill act prohibiti...
327994   sr introduced session senate resolution resolu...
328014   hcr introduced session house concurrent resolu...

In [15]:
#list(compressed_legtext['legtext'][0:8000])[0:2]
compressed_legtext.loc[667269]['texts']

#int(np.where(compressed_legtext.index==804171)[0])


'sb introduced session senate bill act relative training certain board officers nonprofit corporations sponsors sen carson dist sen cataldo dist rep griffin rock rep danielson hills committee public municipal affairs analysis bill requires every publicly supported voluntary corporation ensure chairperson presiding officer board directors designee receives biennial management training explanation matter added current law appears bold italics matter removed current law appears brackets struckthrough matter either new repealed reenacted appears regular type state new hampshire year lord two thousand fifteen act relative training certain board officers nonprofit corporations enacted senate house representatives general court convened purpose purpose act safeguard publicly supported funds amount aggregate units government federal state local received provisions contracts grants provider agreements similar means nonprofit corporations deter waste fraud abuse funds new section voluntary corpo

In [16]:
#prepare the documents by using tfidf vectors:

#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])


stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,2))
stem_counts = stemmed_count_vect.fit_transform(compressed_legtext['texts'][0:amount_train])
stem_counts_test = stemmed_count_vect.transform(compressed_legtext['texts'][amount_train:amount_train+amount_test])

tfidf = TfidfTransformer()
tfidf_vectors = tfidf.fit_transform(stem_counts)
tfidf_vectors_test = tfidf.transform(stem_counts_test)

In [18]:
#do the final pick after, since we only want to run the above cell once
k_best_tfidf = SelectKBest(f_classif, k=500)
final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)
word_indices = k_best_tfidf.get_support(indices=True)
#print(word_indices)

In [26]:
#match up the transformed word vectors with the proper line
print(final_tfidf_features_test.shape)
print(len(compressed_legtext))
compressed_legtext_train = compressed_legtext[0:amount_train]
#add the words as columns titles around here
text_features_df = pd.DataFrame(final_tfidf_features.todense(), index=compressed_legtext_train.index)
text_and_features = pd.concat([compressed_legtext_train, text_features_df],axis=1)
text_and_features.head()

(150, 500)
8908


texts    0    1    2  \
bill_id                                                                     
667269   sb introduced session senate bill act relative...  0.0  0.0  0.0   
132686                                                      0.0  0.0  0.0   
804171   hb introduced session house bill act prohibiti...  0.0  0.0  0.0   
327994   sr introduced session senate resolution resolu...  0.0  0.0  0.0   
328014   hcr introduced session house concurrent resolu...  0.0  0.0  0.0   

                3         4    5    6    7    8  ...  490  491  492  493  494  \
bill_id                                          ...                            
667269   0.000000  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
132686   0.000000  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
804171   0.074516  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
327994   0.000000  0.022445  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   
328014   0.000000  0.000000  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0   

         495       496  497  498  499  
bill_id                                
667269   0.0  0.006523  0.0  0.0  0.0  
132686   0.0  0.000000  0.0  0.0  0.0  
804171   0.0  0.009128  0.0  0.0  0.0  
327994   0.0  0.006213  0.0  0.0  0.0  
328014   0.0  0.002379  0.0  0.0  0.0  

[5 rows x 501 columns]

In [27]:
text_test = text_and_features['texts'].iloc[2:4]
text_test_counts = stemmed_count_vect.transform(text_test)
text_test_tfidf = tfidf.transform(text_test_counts)
text_test_best = k_best_tfidf.transform(text_test_tfidf)
#print(text_test_best)

In [28]:
X_train_all_combined = X_train_all.join(text_and_features.iloc[:,1:])
X_test_all_combined = X_test_all.join(text_and_features.iloc[:,1:])
X_test_all_combined = X_test_all_combined.fillna(value=0)
X_train_all_combined = X_train_all_combined.fillna(value=0)
X_train_all_combined.head()


time_t  sponsor_vec  number_sponsors  chamber_H  chamber_S  \
bill_id                                                               
124543      0.6          0.0              3.0        0.0        0.0   
124543      0.5          0.0              3.0        0.0        0.0   
124543      0.4          0.0              3.0        0.0        0.0   
124543      0.3          0.0              3.0        0.0        0.0   
124543      0.7          0.0              3.0        0.0        0.0   

         bill_type_B  bill_type_CA  bill_type_CR  bill_type_R  \
bill_id                                                         
124543           0.0           1.0           0.0          0.0   
124543           0.0           1.0           0.0          0.0   
124543           0.0           1.0           0.0          0.0   
124543           0.0           1.0           0.0          0.0   
124543           0.0           1.0           0.0          0.0   

         proposed_chamber_H  ...  490  491  492  493  494  495  496  497  498  \
bill_id                      ...                                                
124543                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
124543                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
124543                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
124543                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
124543                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

         499  
bill_id       
124543   0.0  
124543   0.0  
124543   0.0  
124543   0.0  
124543   0.0  

[5 rows x 700 columns]

In [29]:
X_test_all_combined.max().max()

18.0

In [ ]:
#try withouth the words
#try with the state to state
#train the earlier fit with the overall classifier, it compressed_legtextth state to state

In [ ]:
#combine tfidf legtext and the other features:
#legtext_train = final_tfidf_features.todense()
#legtext_train = pd.DataFrame(legtext_train)#*10

#legtext_test = final_tfidf_features_test.todense()
#legtext_test = pd.DataFrame(legtext_test)

#X_train_maybe = np.concatenate([np.array(X_train),legtext_train], axis=1)
#X_test_maybe = np.concatenate([np.array(X_test),legtext_test], axis=1)
#X_train.head()
#X_train[0]

In [30]:
# def perf_metric(y_test, y_pred):
# #Accuracy
#     correct=0
#     total=0
#     acc=0
#     for i in range(len(y_test)):
#         for j in range(len(y_test.iloc[0])):
#             answer = y_test.iloc[i,j] 
#             real = y_pred[i][j]:
#             if answer == real:
#                 correct+=real 
#         total+=1
#     acc=correct/total*100
#     #print(x,y,len(y_pred),len(y_test))
#     #print('Accuracy:', acc)
#     return acc

def perf_metric(y_test, y_pred):
    y_test = np.array(y_test)
    y_pred = np.array(y_pred)
    y_arg_test = np.argwhere(y_test == 1)[:,1]
    print("y_arg_test[0:10] :",y_arg_test[0:10])
    print("len(y_arg_test): ",len(y_arg_test))
    y_arg_pred = np.argwhere(y_pred == 1)
    _pred = [-1]*len(y_arg_test)
    print("Doing got loop...")
    for i, prediction in enumerate(y_arg_pred):
         _pred[prediction[0]] = prediction[1]
    print("Done with for loop")
    y_arg_pred = _pred
    
    print("y_arg_pred[0:10] :",y_arg_pred[0:10])
    print("len(y_arg_pred): ",len(y_arg_pred))
    y_correct = y_arg_test == y_arg_pred
    acc = np.mean(y_correct)
    return acc

In [ ]:
#print(np.array(y_test.iloc[-8]))
#print(predicted[1001])

In [31]:
#train the neural network a bunch
means = []
for i in range(1):
    #print('Training...')
    clf = MultinomialNB(alpha=0)#MLPClassifier(random_state=i+110, max_iter=12)##alpha=0.001,MultinomialNB(alpha=0)#DecisionTreeClassifier()#MultinomialNB()##19 ##
    clf.fit(X_train_all_combined, y_train_all)
    #print('Predicting...')
    predicted = clf.predict(X_test_all_combined)
    predicted_prob = clf.predict_proba(X_test_all_combined)
    #print(predicted_prob[0])
    #train_prediction = clf.predict(X_train)
    #print('Mean Calculation...')
    #new_mean = perf_metric(y_test, predicted)
    new_mean = np.mean(predicted == y_test_all)
    means.append(new_mean)
    print(new_mean)
    #print(np.mean(train_prediction == y_train))
    
mean=np.mean(np.array(means))
print("overall mean:", mean)

0.11352657004830918
overall mean: 0.11352657004830918


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/naive_bayes.py:480: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [32]:
#save the classifier
from joblib import dump, load
dump(clf, _LegOverEventCLF09_20)

['/home/ec2-user/SageMaker/ProjectQuigley/production_notebooks/NH/trained_models/overallevent_clf.joblib']

In [33]:

X_w_preds = X_test_all_combined.copy()
X_w_preds['predictions'] = predicted_prob[:,1]
X_w_preds.loc[894754][['time_t', 'predictions']]

KeyError: 894754

In [ ]:
#X_test_all_combined = X_test_all_combined[X_test_all_combined.columns[0:250]]
#X_test_all_combined.head()

In [ ]:
print(np.mean(y_train_all))
np.mean(y_test_all)

In [34]:
#actualMNBclf = MNBclf
MNBclf = clf
print(type(clf))
#find bill_id from mongoDB:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

bill_id = 1287298
HB_test = X_dum.loc[[bill_id]]#.reshape(1, -1)
print("All data on bill_id", bill_id)
print(X.loc[[bill_id]])
HB_test = k_select.transform(HB_test)
indices = k_select.get_support(indices=True)
print("\n\nUsed data on bill_id", bill_id)
print(X_dum.loc[[bill_id]].iloc[0,indices])

HB_test = np.array(HB_test)

HB_legtext_counts = stemmed_count_vect.transform(X[['texts']].loc[bill_id])
HB_legtext_tfidf = tfidf.transform(HB_legtext_counts)
HB_legtext = k_best_tfidf.transform(HB_legtext_tfidf).todense()
HB_legtext = np.array(HB_legtext)

HB_data = np.concatenate([HB_test, HB_legtext], axis=1)
#print(HB_data)

HB1389_prob = MNBclf.predict_proba(HB_data)
#HB1680_prob = MNBclf.predict_proba(X_dum.loc[1294143])

print(HB1389_prob)
#show all the data
print(X.loc[bill_id])

<class 'sklearn.naive_bayes.MultinomialNB'>
All data on bill_id 1287298
         Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
1287298      119847   2019.919355          0.0     1729  2019       H   

        bill_type                                              title  \
bill_id                                                                
1287298         B  Relative to criminal penalties for not providi...   

        proposed_chamber      primary_name primary_id primary_party  \
bill_id                                                               
1287298                H  Katherine Rogers      14480             D   

         number_sponsors committee_introduced month_introduced  leg_targ  \
bill_id                                                                    
1287298                3                 None               12         0   

                                                

In [ ]:
#save the state-to-state classifier
#StSclf = clf #this has been run and saved

In [35]:
#y_test
def vec_to_state(prediction_vectors, example_output_df):
    index = np.array(example_output_df.columns)
    print('index[0:10]',index[0:10])
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    prediction_vectors = prediction_vectors == 1
    predictions = []
    for prediction in prediction_vectors:
        #print(prediction)
        
        if len(index[prediction]) == 0:
            append_value = [0]
        else:
            append_value = index[prediction][0]
            
        print('append_value:', append_value)
        predictions.append(append_value)
        
    return predictions

def prob_vec_to_n_states(prediction_vector, example_output_df, n=1):
    #return the names, and probabilities in order
    index = np.array(example_output_df.columns)
    
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    ind = np.argpartition(prediction_vector, -n)[-n:]
    sorted_ind = ind[np.argsort(prediction_vector[ind])][::-1]
    
    predictions = []
    for index_s in sorted_ind:
        append_value = index[index_s]
            
        print('append_value:', append_value)
        predictions.append((append_value, prediction_vector[index_s]))
        
    return predictions
    


#pd.get_dummies(X['event'])
def state_to_vec(current_state, example_output_df):
    X_index = np.array(example_output_df.columns)
    indices = current_state #a string

    #for index in indices:
    location = X_index == index
    location = location.astype(int)
        #print(location)
    return(location)
    

In [ ]:
#pd.get_dummies(X[['event', 'month_introduced']]).columns

In [ ]:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

#save the state to state data so we can make a prediction on it next
#sta
#see what is actually in there
#print(X_dum.loc[1294143])
#print(X_dum.loc[1287298])
print(X.loc[1294143])
#prediction_vector = StSclf.predict_proba(X_dum.loc[1294143])[-1]#1294143])
#make an initial prediction of the next step
#print(prob_vec_to_n_states(prediction_vector, y_test, n=5))

#predictions = vec_to_state(prediction_vectors, y_test)
#print(predictions)
#X_dum.head()

In [ ]:
#DTClf has been saved
#DTClf = clf

In [36]:


#print(indices)
#print(word_indices)
type(clf.feature_log_prob_)

numpy.ndarray

In [37]:

num_k = 30
num_words = 1000

#only run this if using MultinomialNB classifier
feature_indices = indices
pass_probs = 10**clf.feature_log_prob_[1] - 10**clf.feature_log_prob_[0]

metadata_feature_pass_probs = {X_dum.columns[feature]:pass_probs[i] for i, feature in enumerate(feature_indices)}
inverted_vocab = stemmed_count_vect.__dict__['vocabulary_']
vocab = {inverted_vocab[key]:key for key in inverted_vocab.keys()}

text_feature_pass_probs = {vocab[feature]:pass_probs[i+num_k] for i, feature in enumerate(word_indices)}
#stemmed_count_vect.__dict__['vocabulary_']['rsa']

all_features = {**metadata_feature_pass_probs, **text_feature_pass_probs}

list_features = list(all_features.keys())

sorted_features = sorted(list_features, key=lambda x: all_features[x], reverse=True)
bad_features = sorted(list_features, key=lambda x: all_features[x], reverse=False)
#print(sorted_features)

#vocab_dict = {indices[i]:X_dum.keys()[indices][i] for i, _ in enumerate(indices)}
#print(vocab_dict)

print("Increase the amount of these features:")
for i, feature in enumerate(sorted_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

print("\n\nDecrease the amount of these features:")
for i, feature in enumerate(bad_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

Increase the amount of these features:
0.0305721285 	 chamber_H
0.0004027675 	 proposed_chamber_S
0.0002660786 	 sponsor_vec
0.0000248906 	 bill_type_B
0.0000084295 	 primary_name_Richard Hinch
0.0000084295 	 constitut
0.0000043141 	 primary_name_Mary Heath
0.0000043141 	 charge st
0.0000042508 	 primary_name_Edward Butler
0.0000042508 	 amending articl
0.0000031189 	 primary_name_Michael Harrington
0.0000031189 	 clarifi
0.0000026180 	 primary_name_Nancy Stiles
0.0000026180 	 clerk hous
0.0000021695 	 primary_name_Peter Leishman
0.0000021695 	 committee transport
0.0000018740 	 primary_name_Raymond Gagnon
0.0000018740 	 congression
0.0000016661 	 primary_name_Michael Cahill
0.0000016661 	 cities ward
0.0000011076 	 primary_name_Norman Major
0.0000011076 	 commissioner depart
0.0000009505 	 primary_name_Kenneth Sheffert
0.0000009505 	 bureau investig
0.0000004914 	 primary_name_Timothy Horrigan
0.0000004914 	 dallesandro dist
0.0000004015 	 primary_name_Janice Schmidt
0.0000004015 	 ap

In [38]:
print(len(predicted))
print()
for i, prediction in enumerate(predicted):
    if i == 50:
        break
    print("real:", y_test[i],"\tpredicted:", predicted_prob[i][1])#, "\tdata:", X_dum.iloc[amount_train:amount_train+amount_test,indices])


414



KeyError: 0